---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA','SW_L','SW_L','PA','SW_H']

In [4]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
def salary_predictions():
    x=pd.DataFrame(nx.degree_centrality(G),index=G.nodes()).T[0]
    y=pd.DataFrame(nx.closeness_centrality(G,normalized=True),index=G.nodes()).T[0]
    z=pd.DataFrame(nx.betweenness_centrality(G,normalized=True),index=G.nodes()).T[0]
    m=[]
    for i in range (len(G.nodes(data=True))):
        m.append(G.nodes(data=True)[i][1])
    X=pd.DataFrame(m)
    X['degree']=x
    X['closeness']=y
    X['beweenness']=z
    test=X[np.isnan(X['ManagementSalary'])==True]
    train=X.dropna()
    features=['Department','degree','closeness','beweenness']
    X_train=train[features]
    X_test=test[features]
    y_train=train['ManagementSalary']
    from sklearn.preprocessing import MinMaxScaler
    X_train_mod=MinMaxScaler().fit_transform(X_train)
    X_test_mod=MinMaxScaler().fit_transform(X_test)
    from sklearn.svm import SVC
    clf=SVC(probability=True)
    clf.fit(X_train_mod,y_train)
    res=clf.predict_proba(X_test_mod)
    
    return pd.Series([x[1] for x in res],index=X_test.index)

In [7]:
salary_predictions()

1       0.320063
2       0.987885
5       1.000000
8       0.251222
14      0.830194
18      0.518635
27      0.717633
30      0.693325
31      0.353832
34      0.142677
37      0.199295
40      0.532911
45      0.507227
54      0.489411
55      0.540839
60      0.366700
62      1.000000
65      0.991470
77      0.094451
79      0.244148
97      0.136013
101     0.182332
103     0.390673
108     0.309132
113     0.916753
122     0.085136
141     0.951639
142     0.999999
144     0.161190
145     0.925854
          ...   
913     0.108520
914     0.104266
915     0.028152
918     0.105325
923     0.035201
926     0.084699
931     0.095933
934     0.072867
939     0.030391
944     0.040287
945     0.035802
947     0.044383
950     0.607430
951     0.055085
953     0.074815
959     0.026226
962     0.028677
963     0.375123
968     0.039324
969     0.036337
974     0.039356
984     0.068102
987     0.078290
989     0.032856
991     0.062718
992     0.030171
994     0.041812
996     0.0318

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [8]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [9]:
def new_connections_predictions():
    
    # Your Code Here
    from sklearn.neighbors import KNeighborsClassifier
    x=list(nx.resource_allocation_index(G))
    y=list(nx.jaccard_coefficient(G))
    z=[(e[0], e[1], len(list(nx.common_neighbors(G, e[0], e[1])))) for e in nx.non_edges(G)]
    x_n=pd.DataFrame(x)
    Y=pd.DataFrame(y)
    Z=pd.DataFrame(z)
    x_n['index']=list(zip(list(x_n[0]), list(x_n[1])))
    Y.drop([0,1],axis=1,inplace=True)
    Z.drop([0,1],axis=1,inplace=True)
    x_n[3]=Z
    x_n[4]=Y
    x_n.index=x_n['index']
    x_n
    features=[2,3,4]
    x_test1=x_n[features]
    X=pd.merge(x_test1,future_connections,how='outer',left_index=True,right_index=True)
    test=X[np.isnan(X['Future Connection'])==True]
    train=X.dropna()
    features=[2,3,4]
    X_train=train[features]
    X_test=test[features]
    y_train=train['Future Connection']
    from sklearn.preprocessing import MinMaxScaler
    X_train_mod=MinMaxScaler().fit_transform(X_train)
    X_test_mod=MinMaxScaler().fit_transform(X_test)
    clf=KNeighborsClassifier()
    clf.fit(X_train_mod,y_train)
    res=clf.predict_proba(X_test_mod)    

    return pd.Series([x[1] for x in res],index=X_test.index)

In [10]:
new_connections_predictions()

(0, 9)          0.0
(0, 19)         0.2
(0, 20)         0.4
(0, 35)         0.0
(0, 38)         0.0
(0, 42)         1.0
(0, 43)         0.0
(0, 50)         0.0
(0, 53)         0.8
(0, 54)         0.0
(0, 62)         0.8
(0, 63)         0.0
(0, 69)         0.0
(0, 72)         0.0
(0, 83)         0.4
(0, 90)         0.0
(0, 91)         0.0
(0, 95)         0.4
(0, 100)        0.0
(0, 106)        1.0
(0, 108)        0.0
(0, 109)        0.0
(0, 110)        0.0
(0, 118)        0.0
(0, 122)        0.0
(0, 131)        0.2
(0, 133)        1.0
(0, 140)        0.4
(0, 149)        0.2
(0, 151)        0.0
               ... 
(988, 989)      0.0
(988, 996)      0.0
(988, 997)      0.2
(988, 1000)     0.0
(988, 1002)     0.0
(989, 994)      0.0
(989, 996)      0.0
(989, 1003)     0.0
(989, 1004)     0.0
(990, 994)      0.0
(990, 998)      0.0
(991, 994)      0.0
(991, 999)      0.0
(991, 1003)     0.0
(992, 994)      0.0
(992, 995)      0.0
(992, 997)      0.0
(992, 1000)     0.0
(993, 1000)     0.0
